<a href="https://colab.research.google.com/github/kwyvanessa/Datathon_2022/blob/main/Datathon_2022_Data_Prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Datathon 2022

## Problem Statement

Preliminary Problem Statement:  What are the three most influential indicators of the gender pay gap 

In [ ]:
# Supress Warnings

import warnings
warnings.filterwarnings('ignore')

# Import the numpy and pandas package

import numpy as np
import pandas as pd
import requests

In [ ]:
url = "https://hdr.undp.org/sites/default/files/2021-22_HDR/HDR21-22_Composite_indices_complete_time_series.csv"
#Use requests package to read csv from url
r = requests.get(url)
open('temp.csv', 'wb').write(r.content)
hdrDF = pd.read_csv('temp.csv')

# Data dictionary for df_all
# https://hdr.undp.org/sites/default/files/2021-22_HDR/HDR21-22_Composite_indices_metadata.xlsx


Glossary of columns<br>
hdi: Human Development Index<br>
le: life expectancy<bdr>
gnipc: Gross National Income per capita<br>
mys: Mean years of schooling<br>
eys: Expected Years of Schooling<br>
gdi:  Gender Development Index<br>
gii: Gender inequality index<br>
mmr: Material mortality rate<br>
abr: Adolescent birth rate <br>
se: population with atleast some secondary ed<br>
pr: share of seats in parliament<br>
lfpr: labor force participation rate<br>
phdi: planetary pressures adjusted HDI<br>
co2_prod: Co2 emissions per capita








In [ ]:
hdr.head()

,iso3,country,hdicode,region,hdi_rank_2021,hdi_1990,hdi_1991,hdi_1992,hdi_1993,hdi_1994,...,mf_2012,mf_2013,mf_2014,mf_2015,mf_2016,mf_2017,mf_2018,mf_2019,mf_2020,mf_2021
0,AFG,Afghanistan,Low,SA,180.0,0.273,0.279,0.287,0.297,0.292,...,1.86,1.88,1.66,1.62,1.66,1.41,1.32,1.38,1.38,1.38
1,AGO,Angola,Medium,SSA,148.0,NaN,NaN,NaN,NaN,NaN,...,4.09,4.53,3.97,3.59,2.79,2.64,2.28,2.18,2.18,2.18
2,ALB,Albania,High,ECA,67.0,0.647,0.629,0.614,0.617,0.624,...,12.44,11.49,13.14,12.61,14.39,14.46,12.85,12.96,12.96,12.96
3,AND,Andorra,Very High,NaN,40.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ARE,United Arab Emirates,Very High,AS,26.0,0.728,0.739,0.742,0.748,0.755,...,49.56,49.68,55.49,59.76,64.95,75.61,65.97,68.95,68.95,68.95


In [ ]:
# Create arrays of column names to use in df slicing
base_cols = ['iso3', 'country', 'hdicode', 'region', 'hdi_rank_2021','rankdiff_hdi_phdi_2021']
shortnames = ['hdi', 'hdi_f','hdi_m','le','le_f','le_m','gnipc','gni_pc_f','gni_pc_m','mys','mys_m','mys_f','eys','eys_f','eys_m','gdi', 'gii','mmr','abr','se_f','se_m','pr_f','lfpr_f','lfpr_m','phdi','co2_prod' ] # a list of the short names of the columns to be selected from the full dataset


In [ ]:
# Define a custom function to create a dataframe of indicators for a given year
def yearslice(base_cols, shortnames, year):
  year_cols = [x + '_' + year for x in shortnames] # an array of column names for filtering
  columns = base_cols + year_cols
  df = hdrDF.loc[:,columns]
  return df


In [ ]:
# Create a sliced dataframe for a given year using the custom function

base_cols = ['iso3', 'country', 'hdicode', 'region', 'hdi_rank_2021','rankdiff_hdi_phdi_2021']
shortnames = ['hdi', 'hdi_f','hdi_m','le','le_f','le_m','gnipc','gni_pc_f','gni_pc_m','mys','mys_m','mys_f','eys','eys_f','eys_m','gdi', 'gii','mmr','abr','se_f','se_m','pr_f','lfpr_f','lfpr_m','phdi','co2_prod' ] # a list of the short names of the columns to be selected from the full dataset
year = '2021'

hdr_21 = yearslice(base_cols,shortnames,year)
hdr_21.head()


,iso3,country,hdicode,region,hdi_rank_2021,rankdiff_hdi_phdi_2021,hdi_2021,hdi_f_2021,hdi_m_2021,le_2021,...,gii_2021,mmr_2021,abr_2021,se_f_2021,se_m_2021,pr_f_2021,lfpr_f_2021,lfpr_m_2021,phdi_2021,co2_prod_2021
0,AFG,Afghanistan,Low,SA,180.0,5.0,0.478,0.364840,0.535611,61.9824,...,0.678,638.0,82.565,6.395730,14.865710,27.215190,14.848,66.515,0.474,0.312376
1,AGO,Angola,Medium,SSA,148.0,10.0,0.586,0.556745,0.616558,61.6434,...,0.537,241.0,138.402,28.163900,51.546170,29.545455,73.968,79.071,0.577,0.675410
2,ALB,Albania,High,ECA,67.0,28.0,0.796,0.799222,0.793605,76.4626,...,0.144,15.0,14.517,95.387505,93.038415,35.714286,50.733,66.154,0.739,1.575754
3,AND,Andorra,Very High,NaN,40.0,NaN,0.858,NaN,NaN,80.3684,...,NaN,NaN,5.887,70.665854,72.422097,46.428571,NaN,NaN,NaN,6.034945
4,ARE,United Arab Emirates,Very High,AS,26.0,-105.0,0.911,0.877132,0.920631,78.7104,...,0.049,3.0,3.085,81.997002,85.596741,50.000000,46.542,88.003,0.518,15.193336


In [ ]:
# Define a custom function to create a timeseries dataframe for a given indicator
def ind_slice(indicator, years):
  ind_cols = [indicator + '_'+ str(x) for x in years]
  columns = base_cols + ind_cols
  df = hdrDF.loc[:,columns]
  return df


In [ ]:
# Create a timeseries DF for a specific indicator (maybe helpful for viz)
indicator = 'hdi'
years = range(2010,2022)
hdi_ts = ind_slice(indicator,years)
hdi_ts.head()

,iso3,country,hdicode,region,hdi_rank_2021,rankdiff_hdi_phdi_2021,hdi_2010,hdi_2011,hdi_2012,hdi_2013,hdi_2014,hdi_2015,hdi_2016,hdi_2017,hdi_2018,hdi_2019,hdi_2020,hdi_2021
0,AFG,Afghanistan,Low,SA,180.0,5.0,0.448,0.456,0.466,0.474,0.479,0.478,0.481,0.482,0.483,0.488,0.483,0.478
1,AGO,Angola,Medium,SSA,148.0,10.0,0.510,0.526,0.541,0.552,0.563,0.582,0.596,0.597,0.595,0.595,0.590,0.586
2,ALB,Albania,High,ECA,67.0,28.0,0.754,0.766,0.778,0.785,0.792,0.795,0.798,0.802,0.806,0.810,0.794,0.796
3,AND,Andorra,Very High,NaN,40.0,NaN,0.848,0.849,0.869,0.864,0.871,0.867,0.871,0.868,0.872,0.873,0.848,0.858
4,ARE,United Arab Emirates,Very High,AS,26.0,-105.0,0.835,0.840,0.846,0.852,0.859,0.865,0.870,0.897,0.909,0.920,0.912,0.911


In [ ]:
https://wbl.worldbank.org/content/dam/wbl/excel/WBL%201971-2022%20Dataset_Updated.xlsx